In [5]:
import os
os.chdir("../../../")

import rrcf
import pandas as pd
import plotly.io as pio

from datetime import  datetime, timedelta
from ai.data_holder.stream_anomaly_detection_data_holder import StreamAnomalyDetectionDataHolder
from ai.data_holder.history_anomaly_detection_data_holder import HistoryAnomalyDetectionDataHolder
from ai.model.stream_anomaly_detection_model import StreamAnomalyDetectionModel
from ai.model.history_anomaly_detection_model import HistoryAnomalyDetectionModel

from ai.interface.model_holder_factory import get_model_holder

from ai.utils.thread_utils import get_stream_history_anomaly_detection_features_with_error_handling
                                  
pd.options.plotting.backend = "plotly"
pio.renderers.default = "vscode"


c:\Users\GMark\Documents\efficientIO\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# dh erstellen
r = {
    "id": "006242f2-aabc-44b1-8986-aa12f6997df5",
    "name": "EIO",
    "data": {
        "chartOptions": [
            {
                "agg": "avg",
                "name": "pac1",
                "scaling": {
                    "max": None,
                    "min": None
                },
                "seriesType": "View",
                "type": "line",
                "unit": "u",
                "var": "pac1"
            }
        ],
        "meta": {
            "controllerMappings": {
                "anomalyScore": "",
                "errorWarning": "0",
                "heartbeat": "0",
                "mpcReady": "0"
              
            },
            "site_id": "",
            "username": "",
            "password": "",
            "errorRule": "197451ec-8329-42c9-bf97-fa945900b35f",
            "warningRule": "cc2fcd5b-b8a0-4d53-9ab8-45d4e722be1e"
        },
        "selectedWidth": "full",
        "threshold": 80,
        "type": "StreamAnomalyDetection"
    },
    "favorite": False,
    "collection_id": "082d51be-d4bd-4bbd-bac0-cd2cbea54730",
    "project_id": "a4a2a88c-9803-4431-8fe3-d021b814719a",
    "created_at": {}
}
dh = StreamAnomalyDetectionDataHolder.from_dict(r)


# Scaled Multifeature

In [3]:
# SET "NOW"-DATETIME TO BE THE SAME FOR ALL EXAMPLES

now = datetime(2023, 5, 1, 12, 0)
training_timedelta_seconds = 1600*60 # 1 day


model_holder = get_model_holder(dh)

# training data from "now"-24h to "now"
df = model_holder.get_dh().get_data(int(now.timestamp()) - training_timedelta_seconds, int(now.timestamp())+60, add_minutes_of_day=True)

model_holder.set_model(StreamAnomalyDetectionModel.train(df, sample_size=1600, shingle_size=4)) # ADJUSTMENT: Sample Size Increased
model_holder.set_status(status="running", ready=1, error=0, training=False)

df.head()
df["pac1"].plot()

c:\Users\GMark\Documents\efficientIO\.venv\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [4]:
# new version does not load so much data

def get_value(ts0, hour):

    ts_start = int(ts0- 60*8) # ADJUSTMENT: less data loaded
    ts_end = int(ts0+30) # COMMENT: if ts0 close to zero point[-1] == point[-2] due to None Value on request, values arrive in seconds 
    model_holder._status.ready = 1
    x, _ = get_stream_history_anomaly_detection_features_with_error_handling(model_holder, ts_start,ts_end, shingle_size=4, add_minute_of_day=hour)

    return x

In [5]:
from tqdm import tqdm

end = now + timedelta(hours=24*1)

start = int(now.timestamp())

result = []

for ts0 in tqdm(range(start, int(end.timestamp()),60)):
    x = get_value(ts0, hour=True)
    score = model_holder.get_anomaly_score(x)
    model_holder.insert_point(x)
    model_holder.set_anomaly_score(score)
    model_holder.set_status(ready=2)

    result.append({"score":score, "value1":x[3], "value2":x[7], "value3":x[-1], "time":datetime.fromtimestamp(ts0)})
    #print({"value":x, "score":score, "time":datetime.fromtimestamp(ts0)})

  0%|          | 0/1440 [00:00<?, ?it/s]

100%|██████████| 1440/1440 [02:16<00:00, 10.53it/s]


In [6]:
df = pd.DataFrame(result)
df.set_index(df["time"], inplace=True)
df.drop(["time"], inplace=True, axis=1)
df.plot()

c:\Users\GMark\Documents\efficientIO\.venv\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



# History AD

In [2]:
import os
os.chdir("../../../")

import rrcf
import pandas as pd
import plotly.io as pio

from datetime import  datetime, timedelta
from ai.data_holder.stream_anomaly_detection_data_holder import StreamAnomalyDetectionDataHolder
from ai.data_holder.history_anomaly_detection_data_holder import HistoryAnomalyDetectionDataHolder
from ai.model.stream_anomaly_detection_model import StreamAnomalyDetectionModel
from ai.model.history_anomaly_detection_model import HistoryAnomalyDetectionModel

from ai.interface.model_holder_factory import get_model_holder

from ai.utils.thread_utils import get_stream_history_anomaly_detection_features_with_error_handling
                                  
pd.options.plotting.backend = "plotly"
pio.renderers.default = "vscode"


c:\Users\GMark\Documents\efficientIO\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_start = int(datetime(2023,10,2,0,0).timestamp())
train_end = int(datetime(2023,10,4,0,0).timestamp())


r={
        "id": "18e9282d-c525-448b-872d-4fd850f321ea",
        "name": "Hist",
        "data": {
            "chartOptions": [
                {
                    "agg": "avg",
                    "name": "pac1",
                    "scaling": {
                        "max": None,
                        "min": None
                    },
                    "seriesType": "View",
                    "type": "line",
                    "unit": "u",
                    "var": "pac1"
                }
            ],
            "meta": {
                "controllerMappings": {
                    "anomalyScore": "",
                    "endDate": train_end,
                    "errorWarning": "0",
                    "heartbeat": "0",
                    "mpcReady": "0",
                    "startDate": train_start
                },
                "site_id": "",
                "username":"",
                "password":"",
                "errorRule": "6eb948db-8ffb-410a-816f-167a1b238245",
                "warningRule": "58ab9195-7a9a-44c8-99a1-4e7e14de39ba"
            },
            "selectedWidth": "full",
            "threshold": 80,
            "type": "HistoryAnomalyDetection"
        },
        "favorite": False,
        "collection_id": "082d51be-d4bd-4bbd-bac0-cd2cbea54730",
        "project_id": "a4a2a88c-9803-4431-8fe3-d021b814719a",
        "created_at": {}
    }

dh = HistoryAnomalyDetectionDataHolder.from_dict(r)

In [4]:
# Scaled Multifeature
# SET "NOW"-DATETIME TO BE THE SAME FOR ALL EXAMPLES

model_holder = get_model_holder(dh)

df = model_holder.get_dh().get_data(model_holder.get_start(), model_holder.get_end(), add_minute_of_day=True)

model_holder.set_model(HistoryAnomalyDetectionModel.train(df, sample_size=1600, shingle_size=4))
model_holder.set_status(status="running", ready=1, error=0, training=False)

df.head()
df["pac1"].plot()


c:\Users\GMark\Documents\efficientIO\.venv\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [5]:

# new version does not load so much data

def get_value(ts0, add_minute_of_day):

    ts_start = int(ts0- 60*20) # ADJUSTMENT: less data loaded
    ts_end = int(ts0+30) # COMMENT: if ts0 close to zero point[-1] == point[-2] due to None Value on request, values arrive in seconds 
    model_holder._status.ready = 1
    x, _ = get_stream_history_anomaly_detection_features_with_error_handling(model_holder, ts_start,ts_end, shingle_size=4, add_minute_of_day=add_minute_of_day)

    return x

In [14]:

from tqdm import tqdm

now = datetime(2023, 5, 1, 12, 0)


end = now + timedelta(hours=24*5)

start = int(now.timestamp())

result = []

for ts0 in tqdm(range(start, int(end.timestamp()),60)):
    x = get_value(ts0, add_minute_of_day=True)
    try:
        score = model_holder.get_anomaly_score(x)
    except:
        print(x)
    #model_holder.insert_point(x)
    model_holder.set_anomaly_score(score)
    model_holder.set_status(ready=2)

    result.append({"score":score, "value1":x[3], "value2":x[7], "value3":x[-1], "time":datetime.fromtimestamp(ts0)})
    #print({"value":x, "score":score, "time":datetime.fromtimestamp(ts0)})
    


  1%|          | 154/14400 [00:22<33:20,  7.12it/s]

[19.17       18.94       15.54       16.25        0.19735301  0.19561929
  0.19389136  0.19216926  0.101999    0.10332333  0.10465521  0.10599462]


  2%|▏         | 257/14400 [00:33<22:26, 10.50it/s]

In [13]:
df = pd.DataFrame(result)
df.set_index(df["time"], inplace=True)
df.drop(["time"], inplace=True, axis=1)
df.plot()

c:\Users\GMark\Documents\efficientIO\.venv\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

